In [1]:
import sys
import os
import math
import random
import bisect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import subprocess
import re
import tempfile
import itertools
import torch

from typing import List, Tuple
from operator import itemgetter 
# import qgrid

In [2]:
BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')

In [3]:
from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
# from nebula_api.mdmmt_api import mdmmt_api
from nebula_api.vlmapi import VLM_API

In [4]:
import nebula_api.playground_api as pg_api

In [5]:
nre = NRE_API()
api = RemoteAPIUtility()
# mdmmt = mdmmt_api.MDMMT_API()
vlm = VLM_API()

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14']
Milvus Collection Loaded:  vcomet_mdmmt_embedded_event
Milvus Collection Loaded:  vcomet_mdmmt_embedded_place
Milvus Collection Loaded:  vcomet_mdmmt_embedded_actions
INFO:tensorflow:Restoring parameters from /app/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transf

INFO:tensorflow:Restoring parameters from /app/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transf

Available class names: ['clip_vit', 'clip_rn', 'mdmmt_max', 'mdmmt_mean', 'mdmmt_legacy']


In [6]:
def get_vlmtest_data_from_db():
    results = []
    query = '''FOR doc IN nebula_lsmdc_groundtruth
               COLLECT mid = doc.movie_id, elem = doc.scene_element INTO groups = [doc.sentence, doc.quality]
                RETURN { 
                    "name": CONCAT_SEPARATOR("_",mid,elem),
                    "mid": mid,
                    "elem": elem,
                    "sentences": groups
                }'''
    cursor = nre.db.aql.execute(query)
    for doc in cursor:
        results.append(doc)
    return (results)

def compute_batch_scores(video_emb: torch.Tensor, texts: List[str], normalize=True, **kwargs) -> List[float]:    
    emb_batch = vlm.encode_text(texts, **kwargs)
    if type(emb_batch) == list:
        emb_batch = torch.stack(emb_batch,axis=0)
    if normalize:
        video_emb = video_emb / video_emb.norm(2)
        # print("normalized video norm: {}".format(video_emb.norm(2)))
        n = (emb_batch * emb_batch).sum(axis=1).sqrt()
        emb_batch = emb_batch / n.unsqueeze(1).expand_as(emb_batch)
        # print("normalized text norms:")
        # for emb in emb_batch:
        #     print(emb.norm(2))                        
    return (video_emb.unsqueeze(0).expand_as(emb_batch)*emb_batch).sum(dim=1).cpu().numpy()

def get_movie_url(mid):
    query = 'FOR doc IN Movies FILTER doc._id == "{}" RETURN doc'.format(mid)
    cursor = api.db.aql.execute(query)
    url_prefix = "http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/"
    url_link = ''
    for doc in cursor:
        url_link = url_prefix+doc['url_path']
        url_link = url_link.replace(".avi", ".mp4")   
    return url_link


def plot_index(i,df1, s=70):
    elem = all_elems[i]
    mid = all_movies[i]
    info = api.get_movie_info(mid)
    title = info['description']
    df = df1.loc[all_names[i],:,:]
    print(get_movie_url(mid))
    print([(j+1,x) for (j,x) in enumerate(all_sentences[i])])
    plt.figure(figsize=(40, 18))
    plt.ylim(0,0.5)
    sns.scatterplot(data = df.reset_index(), x = 'index', y = 'actual', hue='golden', style='vlm_type',s=s,
               palette={1:'r', 2:'b', 3:'g'}).set(title="{}/{} - {}".format(mid,elem,title))


In [7]:
vlm_types = ['mdmmt_legacy', 'mdmmt_max', 'mdmmt_mean']

In [8]:
data = get_vlmtest_data_from_db()

In [18]:
all_names = []
all_movies = []
all_elems = []
all_goldens = []
all_sentences = []
for i,d in enumerate(data):
    if i not in [4,6,7,9,10]:      # 6 has 1 sentence, others are too short
        all_names.append(d['name'])
        all_movies.append(d['mid'])
        all_elems.append(d['elem'])
        sentences = list(zip(*sorted(d['sentences'], key=lambda x: -x[1])))
        all_sentences.append(sentences[0])
        all_goldens.append(sentences[1])

In [19]:
a = np.expand_dims(np.array(all_goldens),1)
newshape = list(a.shape)
newshape[1]=len(vlm_types)
goldens = np.broadcast_to(a,newshape)

In [20]:
# mid = all_movies[2]
# elem = all_elems[2]
# s = all_sentences[2]
# encoding_method = vlm_types[0]
# video_emb = vlm.encode_video(mid,elem,class_name=encoding_method).squeeze().cuda()
# # text_emb = vlm.encode_text([s], class_name=encoding_method)

# emb_batch = vlm.encode_text(s,class_name=encoding_method)
# if type(emb_batch) == list:
#         emb_batch = torch.stack(emb_batch,axis=0)

# n = (emb_batch * emb_batch).sum(axis=1).sqrt()
# n.unsqueeze(1).expand_as(emb_batch)[6]

# compute_batch_scores(video_emb,s, normalize=True, class_name=encoding_method)

# df1.loc[all_names[2],encoding_method,:]

# (video_emb*video_emb).sum().sqrt()
# [x['name'] for x in data].index('Movies/114207908_1')

ValueError: 'Movies/114207908_1' is not in list

In [21]:
all_scores = []
for encoding_method in vlm_types:
    method_scores = []
    for mid, elem, sentences, in zip(all_movies,all_elems, all_sentences):        
        print("Trying to compute embedding for movie {}, element {}, class {}".format(mid,elem,encoding_method))
        video_emb = vlm.encode_video(mid,elem,class_name=encoding_method).squeeze().cuda()
        method_scores.append(compute_batch_scores(video_emb, sentences,class_name=encoding_method))
    all_scores.append(np.stack(method_scores,axis=0))       

Trying to compute embedding for movie Movies/114206337, element 0, class mdmmt_legacy
http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/1015_27_Dresses_00_38_02_757-00_38_08_213.mp4
Movie info: {'arango_id': 'Movies/114206337', 'description': '1015_27_Dresses_00_38_02_757-00_38_08_213', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f5ebaf4686c547a0bd8bc3859933a091', 'mdfs': [[2, 36, 71], [75, 100, 125], [129, 129, 129]], 'scene_elements': [[0, 73], [73, 127], [127, 131]]}
fn path: /tmp/video_file.mp4
Trying to compute embedding for movie Movies/114206337, element 1, class mdmmt_legacy
http://ec2-18-159-140-240.eu-central-1.compute.amazonaws.com:7000/static/development/1015_27_Dresses_00_38_02_757-00_38_08_213.mp4
Movie info: {'arango_id': 'Movies/114206337', 'description': '1015_27_Dresses_00_38_02_757-00_38_08_213', 'fps': 23, 'width': 1920, 'height': 1080, 'last frame': 300, 'movie_id': 'f5ebaf4686c547a0bd8bc3859933a091',

In [22]:
actual = np.stack(all_scores,axis=0).transpose([1,0,2])
init = np.stack([actual,goldens],axis=3)

In [23]:
mindex = pd.MultiIndex.from_product([all_names, vlm_types,range(1,11)], names=["movie", "vlm_type", "index"])
df1 = pd.DataFrame(init.reshape(-1,2),index=mindex,columns=['actual', 'golden'])

In [ ]:
df = df1.loc[all_names[5],:,:]
plt.figure(figsize=(15, 8))
g = sns.scatterplot(data = df.reset_index(), x = 'index', y = 'actual', hue='golden', style='vlm_type', s=70,
               palette={1:'r', 2:'b', 3:'g'})

In [35]:
plot_index(11,df1,s=140)

IndexError: list index out of range

In [ ]:
api.get_movie_info('Movies/114208744')

In [ ]:
df2 = df1.unstack()

In [ ]:
df2.unstack().columns

In [ ]:
# Nested dictionary to convert it into multiindex dataframe
nested_dict = {'India': {'State': ['Maharashtra', 'West Bengal',
								'Uttar Pradesh', 'Bihar', 'Karnataka'],
						'Capital': ['Mumbai', 'Kolkata', 'Lucknow',
									'Patna', 'Bengaluru']},

			'America': {'State': ['California', 'Florida', 'Georgia',
									'Massachusetts', 'New York'],
						'Capital': ['Sacramento', 'Tallahassee', 'Atlanta',
									'Boston', 'Albany']}}

reformed_dict = {}
for outerKey, innerDict in nested_dict.items():
	for innerKey, values in innerDict.items():
		reformed_dict[(outerKey, innerKey)] = values

# Display multiindex dataframe
multiIndex_df = pd.DataFrame(reformed_dict)
multiIndex_df


In [ ]:
reformed_dict